<a href="https://colab.research.google.com/github/Kzis/mrt-is/blob/master/Prep_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv" , nrows=10000)
df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv")

In [4]:
df.shape

(17240769, 24)

In [5]:
df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


In [6]:
def get_data_by_month(month):
  return df.loc[df['ThaiMonthName'] == month]

def get_transaction_df(month_df):
  df_month_transaction_date_sum = month_df.TransactionDate.value_counts().rename_axis('TransactionDate').reset_index(name='TransactionCounts')
  df_month_transaction_date_sum.sort_values('TransactionDate')
  return df_month_transaction_date_sum

In [31]:
def get_entry_total_station_by_date(month_df , month_transaction_df , col_name):
  date_list = month_transaction_df['TransactionDate'].tolist()
  col_list = month_df[col_name].unique().tolist()
  output = []
  for date in date_list:
    # print("Process : {}".format(date))
    for col_val in col_list:
      lst = []
      subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
      column_count = subset_df.value_counts()
      lst.extend([date,"Entry", col_val, column_count.index[0]])
      output.append(lst)
      # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
  # print(output)
  df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
  return df  

def get_exit_total_station_by_date(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    output = []
    for date in date_list:
      # print("Process : {}".format(date))
      for col_val in col_list:
        lst = []
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.extend([date,"Exit", col_val, column_count.index[0]])
        output.append(lst)
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
    # print(output)
    df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    return df  


In [7]:
jan_df = get_data_by_month("มกราคม")
jan_df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


In [33]:
transaction_df = get_transaction_df(jan_df)
transaction_df.head()

,TransactionDate,TransactionCounts
0,2017-01-25,37135
1,2017-01-26,36485
2,2017-01-13,36171
3,2017-01-24,36050
4,2017-01-20,35898


# Prep ข้อมูล เข้าและออกทั้งหมดของแต่ละสถานี

In [28]:
 entry_df = get_entry_total_station_by_date(jan_df , transaction_df , "StationName")
 entry_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Entry,Sam Yan,1328
1,2017-01-25,Entry,Si Lom,2865
2,2017-01-25,Entry,Lumphini,1448
3,2017-01-25,Entry,Khlong Toei,308
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400
...,...,...,...,...
553,2017-01-01,Entry,Phahon Yothin,352
554,2017-01-01,Entry,Chatuchak Park,359
555,2017-01-01,Entry,Kamphaeng Phet,233
556,2017-01-01,Entry,Bang Sue,186


In [32]:
exit_df = get_exit_total_station_by_date(jan_df , transaction_df , "Right_StationName")
exit_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Exit,Hua Lamphong,1338
1,2017-01-25,Exit,Sam Yan,1299
2,2017-01-25,Exit,Si Lom,2720
3,2017-01-25,Exit,Lumphini,1621
4,2017-01-25,Exit,Khlong Toei,289
...,...,...,...,...
553,2017-01-01,Exit,Lat Phrao,309
554,2017-01-01,Exit,Phahon Yothin,342
555,2017-01-01,Exit,Chatuchak Park,250
556,2017-01-01,Exit,Kamphaeng Phet,151


In [34]:
jan_transaction_df = entry_df.append(exit_df) 
jan_transaction_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Entry,Sam Yan,1328
1,2017-01-25,Entry,Si Lom,2865
2,2017-01-25,Entry,Lumphini,1448
3,2017-01-25,Entry,Khlong Toei,308
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400
...,...,...,...,...
553,2017-01-01,Exit,Lat Phrao,309
554,2017-01-01,Exit,Phahon Yothin,342
555,2017-01-01,Exit,Chatuchak Park,250
556,2017-01-01,Exit,Kamphaeng Phet,151


# Export Data 

In [35]:
jan_transaction_df.to_excel (r'Jan_Summary.xlsx', index = False, header=True)